In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import csv

In [2]:
samples = 1000
test_samples = 100
train_dataset = './ffnn_dataset/train_dataset.csv'
test_dataset = './ffnn_dataset/test_dataset.csv'

def write_dataset(samples, test_samples, train_dir, test_dir):
    up = [i for i in range(10)]
    down = [9-i for i in range(10)]

    data = []
    label = []
    for i in range(samples):
        data.append(up)
        data.append(down)
        label.append([1])
        label.append([0])

    with open(train_dataset, 'w') as csvfile:
        writer = csv.writer(csvfile)
        for i in range(samples-test_samples):
            writer.writerow(label[i] + data[i])
        print('train data is written')

    with open(test_dataset, 'w') as csvfile:
        writer = csv.writer(csvfile)
        for i in range(test_samples):
            writer.writerow(label[i] + data[i])
        print('test data is written')
        
#write_dataset(1000, 100, train_dataset, test_dataset)

train data is written
test data is written


- dataset framwork : tf.contrib.data.TextLineDataset

In [5]:
#decorate, shuffle = seed
dataset = tf.contrib.data.TextLineDataset(train_dataset).batch(32)
#dataset = dataset.shuffle(7777)
#한번 스캔하고 끝내지 말고 반복해서 읽어라 에러내지말고
dataset = dataset.repeat(100)

itr = dataset.make_one_shot_iterator()

batch = itr.get_next()

#decoding
decoded_batch = tf.decode_csv(batch, record_defaults=[[0]]*11)

#rank를 같게 만들어 준다.(reshape, expand_dims 같은 방법)
#label = decoded_batch[0]
label = tf.reshape(decoded_batch[0], [-1, 1])
#label = tf.expand_dims(decoded_batch[0], axis=-1)

#column을 사용
#row_feature = tf.stack(decoded_batch[1:], axis=0)
feature = tf.stack(decoded_batch[1:], axis=1)

#타입 변경
label = tf.cast(label, tf.float32)
feature = tf.cast(feature, tf.float32)

print(label.dtype)
print(feature.dtype)

Instructions for updating:
Use `tf.data.TextLineDataset`.
<dtype: 'float32'>
<dtype: 'float32'>


In [6]:
layer1 = tf.layers.dense(feature, units=15)
layer2 = tf.layers.dense(layer1, units=10)
layer3 = tf.layers.dense(layer2, units=5)
layer4 = tf.layers.dense(layer3, units=3)
out = tf.layers.dense(layer4, units=1)

print('layer1 shape {}',format(layer1.shape))
print('layer2 shape {}',format(layer2.shape))
print('layer3 shape {}',format(layer3.shape))
print('layer4 shape {}',format(layer4.shape))
print('out shape {}',format(out.shape))

layer1 shape {} (?, 15)
layer2 shape {} (?, 10)
layer3 shape {} (?, 5)
layer4 shape {} (?, 3)
out shape {} (?, 1)


- label 이랑 out이랑 shape이 같다

In [7]:
loss = tf.losses.sigmoid_cross_entropy(label, out)
train_op = tf.train.GradientDescentOptimizer(1e-2).minimize(loss)

In [8]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    _loss, _ = sess.run([loss, train_op])
    for i in range(1001):
        _loss, _ = sess.run([loss, train_op])
        if i%100 == 0:
            print('step: {}, loss: {}'.format(i, _loss))


step: 0, loss: 0.0552276112139225
step: 100, loss: 0.0020712092518806458
step: 200, loss: 0.000978351105004549
step: 300, loss: 0.0006241615628823638
step: 400, loss: 0.00045241316547617316
step: 500, loss: 0.0003520671743899584
step: 600, loss: 0.00028668655431829393
step: 700, loss: 0.00024090615625027567
step: 800, loss: 0.0002071705530397594
step: 900, loss: 0.0001813407288864255
step: 1000, loss: 0.00016096752369776368


## train and test

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import csv

In [2]:
samples = 1000
test_samples = 100
train_dataset = './ffnn_dataset/train_dataset.csv'
test_dataset = './ffnn_dataset/test_dataset.csv'

def write_dataset(samples, test_samples, train_dir, test_dir):
    up = [i for i in range(10)]
    down = [9-i for i in range(10)]

    data = []
    label = []
    for i in range(samples):
        data.append(up)
        data.append(down)
        label.append([1])
        label.append([0])

    with open(train_dataset, 'w') as csvfile:
        writer = csv.writer(csvfile)
        for i in range(samples-test_samples):
            writer.writerow(label[i] + data[i])
        print('train data is written')

    with open(test_dataset, 'w') as csvfile:
        writer = csv.writer(csvfile)
        for i in range(test_samples):
            writer.writerow(label[i] + data[i])
        print('test data is written')
        
write_dataset(1000, 100, train_dataset, test_dataset)

train data is written
test data is written


In [3]:
trainset = tf.contrib.data.TextLineDataset(train_dataset).batch(10)
testset = tf.contrib.data.TextLineDataset(test_dataset).batch(10)

trainset = trainset.repeat(100)
testset = testset.repeat(999999)

train_itr = trainset.make_one_shot_iterator()
test_itr = testset.make_one_shot_iterator()

train_batch = train_itr.get_next()
test_batch = test_itr.get_next()

train_batch = tf.decode_csv(train_batch, [[0]]*11)
test_batch = tf.decode_csv(test_batch, [[0]]*11)

train_itr = trainset.make_one_shot_iterator()
test_itr = testset.make_one_shot_iterator()

train_batch = train_itr.get_next()
test_batch = test_itr.get_next()

train_decoded_batch = tf.decode_csv(train_batch, record_defaults=[[0]]*11)
test_decoded_batch = tf.decode_csv(test_batch, record_defaults=[[0]]*11)

train_label = tf.reshape(train_decoded_batch[0], [-1, 1])
test_label = tf.reshape(test_decoded_batch[0], [-1, 1])

train_feature = tf.stack(train_decoded_batch[1:], axis=1)
test_feature = tf.stack(test_decoded_batch[1:], axis=1)


train_label = tf.cast(train_label, tf.float32)
train_feature = tf.cast(train_feature, tf.float32)

test_label = tf.cast(test_label, tf.float32)
test_feature = tf.cast(test_feature, tf.float32)


Instructions for updating:
Use `tf.data.TextLineDataset`.


In [4]:
def bin_model(feature, activation=None, reuse=False):
    layer1 = tf.layers.dense(feature, units=15, activation=activation, reuse=reuse, name='layer1')
    layer2 = tf.layers.dense(layer1, units=10, activation=activation, reuse=reuse, name='layer2')
    layer3 = tf.layers.dense(layer2, units=5, activation=activation, reuse=reuse, name='layer3')
    layer4 = tf.layers.dense(layer3, units=3, activation=activation, reuse=reuse, name='layer4')
    out = tf.layers.dense(layer4, units=1, reuse=reuse, name='out')
    print('layer1 shape {}',format(layer1.shape))
    print('layer2 shape {}',format(layer2.shape))
    print('layer3 shape {}',format(layer3.shape))
    print('layer4 shape {}',format(layer4.shape))
    print('out shape {}',format(out.shape))
    return out


train_out = bin_model(train_feature)
#reuse=True = 기존모델 그대로 사용해라
test_out = bin_model(test_feature, reuse=True)
print(train_out, test_out)

layer1 shape {} (?, 15)
layer2 shape {} (?, 10)
layer3 shape {} (?, 5)
layer4 shape {} (?, 3)
out shape {} (?, 1)
layer1 shape {} (?, 15)
layer2 shape {} (?, 10)
layer3 shape {} (?, 5)
layer4 shape {} (?, 3)
out shape {} (?, 1)
Tensor("out/BiasAdd:0", shape=(?, 1), dtype=float32) Tensor("out_2/BiasAdd:0", shape=(?, 1), dtype=float32)


In [5]:
for v in tf.trainable_variables():
    print(v)

<tf.Variable 'layer1/kernel:0' shape=(10, 15) dtype=float32_ref>
<tf.Variable 'layer1/bias:0' shape=(15,) dtype=float32_ref>
<tf.Variable 'layer2/kernel:0' shape=(15, 10) dtype=float32_ref>
<tf.Variable 'layer2/bias:0' shape=(10,) dtype=float32_ref>
<tf.Variable 'layer3/kernel:0' shape=(10, 5) dtype=float32_ref>
<tf.Variable 'layer3/bias:0' shape=(5,) dtype=float32_ref>
<tf.Variable 'layer4/kernel:0' shape=(5, 3) dtype=float32_ref>
<tf.Variable 'layer4/bias:0' shape=(3,) dtype=float32_ref>
<tf.Variable 'out/kernel:0' shape=(3, 1) dtype=float32_ref>
<tf.Variable 'out/bias:0' shape=(1,) dtype=float32_ref>


In [6]:
#모델 세이브 saver()
saver = tf.train.Saver()

In [7]:
loss = tf.losses.sigmoid_cross_entropy(train_label, train_out)
train_op = tf.train.GradientDescentOptimizer(1e-2).minimize(loss)

pred = tf.nn.sigmoid(test_out)
accuracy = tf.metrics.accuracy(test_label, tf.round(pred))

In [8]:
#graph 그리기
tf.summary.scalar('loss', loss)

merged = tf.summary.merge_all()

In [9]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())#accuracy 계속 이동 평균구하기 때문, 내부적인 variable 필요
    writer = tf.summary.FileWriter('./ffnn_logs/', sess.graph)
    for i in range(1001):
        _, _loss, _acc, _summ = sess.run([train_op, loss, accuracy, merged])
        writer.add_summary(_summ, i)
        if i%100 == 0:
            _pred = sess.run(pred)
            print('step: {}, loss: {}, acc: {}'.format(i, _loss, _acc[0]))
            
    saver.save(sess, './ffnn_logs/ffnn')

step: 0, loss: 0.0072141303680837154, acc: 0.0
step: 100, loss: 0.0003634498571045697, acc: 1.0
step: 200, loss: 0.00020566396415233612, acc: 1.0
step: 300, loss: 0.00014801087672822177, acc: 1.0
step: 400, loss: 0.00011711772822309285, acc: 1.0
step: 500, loss: 9.749164746608585e-05, acc: 1.0
step: 600, loss: 8.376597543247044e-05, acc: 1.0
step: 700, loss: 7.355816342169419e-05, acc: 1.0
step: 800, loss: 6.563468923559412e-05, acc: 1.0
step: 900, loss: 5.928733662585728e-05, acc: 1.0
step: 1000, loss: 5.4078391258371994e-05, acc: 1.0


## 모델 불러오기

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, './ffnn_logs/ffnn')
    _pred = sess.run(pred)
    print(_pred)

## Graph 확인
- $tensorboard --logdir=./ffnn_logs
- http://0.0.0.0:6006/